<a href="https://colab.research.google.com/github/fmind/genv/blob/main/GenV_Generative_AI_for_Video_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP

## Google Cloud Platform (GCP)

1.  **Create or Select a GCP Project:** Ensure you have a Google Cloud project. If not, create one.
2.  **Enable Vertex AI API:** Within your chosen project, navigate to the APIs & Services section and enable the "Vertex AI API".
3.  **Configure Project ID Secret:** Add your GCP `PROJECT_ID` as a Colab secret named `GENV_PROJECT_ID`. This notebook will use this secret for authentication.


## Google Cloud Storage (GCS)

1.  **Create a GCS Bucket:** Create a Cloud Storage bucket within your GCP project. This bucket will be used to temporarily store video files for processing by the Vertex AI API. Update the `BUCKET` configuration variable with the name of your bucket.

## Google Drive

1.  **Authorize Access:** You will be prompted to authorize this notebook to access your Google Drive. This is necessary to read the video files specified in the `VIDEOS_PATH` configuration. Ensure your Meet recordings are located at this path.

# CONFIGS

In [ ]:
# @title Generative AI

MODEL = "gemini-2.0-flash-lite" # @param {"type":"string"}
TEMPERATURE = 0.0 # @param {"type":"slider","min":0,"max":2,"step":0.1}

In [ ]:
# @title Google Cloud

PROJECT_ID = "" # @param {"type":"string"}
LOCATION = "us-central1" # @param {"type":"string"}
BUCKET = "genv-videos" # @param {"type": "string"}

In [ ]:
# @title App

SINCE_DAYS = 7 # @param {"type":"integer"}
VIDEOS_PATH = "/content/drive/MyDrive/Meet Recordings" # @param {"type":"string"}
DOWNLOAD = True # @param {"type":"boolean"}

# IMPORTS

## Internal

In [ ]:
import io
import json
import string
import textwrap
import typing as T
import datetime as dt
from pathlib import Path

## External

In [ ]:
import pydantic as pdt
from google import genai
from IPython import display
from google.cloud import storage
from google.genai import types as GT
from google.colab import userdata, drive, auth, files

# SECRETS

## Project

In [ ]:
PROJECT_ID = PROJECT_ID or userdata.get('GENV_PROJECT_ID')

# SERVICES

## Authentication

In [ ]:
auth.authenticate_user(project_id=PROJECT_ID)

## Storage

In [ ]:
storage_client = storage.Client()

## Gen AI

In [ ]:
genai_client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

## Drive

In [ ]:
drive.mount('/content/drive')

## Bucket

In [ ]:
bucket = storage_client.bucket(BUCKET)

# CONTENTS

## Filters

In [ ]:
today = dt.date.today()
since = today - dt.timedelta(days=SINCE_DAYS)
since

## Paths

In [ ]:
root = Path(VIDEOS_PATH)
paths = []
for path in root.glob('*'):
    if path.is_file():
        modification_time = dt.datetime.fromtimestamp(path.stat().st_mtime).date()
        if modification_time >= since:
            paths.append(path)
paths

## Blobs

In [ ]:
blobs = {}
for path in paths:
    try:
        blob = bucket.blob(f"{path.stem}.mp4")
        if not blob.exists():
            blob.upload_from_filename(path)
            print(f'Path "{path.name}" uploaded to blob "{blob.name}".')
        else:
            print(f'Blob "{blob.name}" already exists.')
        blobs[path] = blob
    except Exception as error:
        print(f"[ERROR] {error}")
blobs

# ANALYSIS

## Data Classes

In [ ]:
# --- Define the structure for individual pieces of information ---

class QuestionAnswer(pdt.BaseModel):
    """Represents a question asked during the meeting and its corresponding answer."""
    question: str = pdt.Field(description="The specific question that was asked.")
    answer: str = pdt.Field(description="The answer provided during the meeting.")

class ProjectInfo(pdt.BaseModel):
    """Represents a project discussed during the meeting."""
    name: str = pdt.Field(description="The name or identifier of the project mentioned.")
    details: str = pdt.Field(description="A brief summary of the status update, discussion points, or decisions related to this project from the meeting.")

class ActionItem(pdt.BaseModel):
    """Represents an action item or task assigned during the meeting."""
    task: str = pdt.Field(description="The description of the action item or task.")
    owner: T.Optional[str] = pdt.Field(default=None, description="The person or team assigned responsibility for the task, if specified.")
    deadline: T.Optional[str] = pdt.Field(default=None, description="The deadline mentioned for the task, if specified (e.g., 'Next week', 'YYYY-MM-DD').")

class DecisionSuggestion(pdt.BaseModel):
    """Represents a key decision made or a significant suggestion proposed during the meeting."""
    item: str = pdt.Field(description="The core decision made or suggestion proposed.")
    type: T.Literal["Decision", "Suggestion"] = pdt.Field(description="Indicates whether this is a Decision or a Suggestion.")
    details: T.Optional[str] = pdt.Field(default=None, description="Any relevant context, rationale, or elaboration provided.")


# --- Define the main structure for the overall meeting insights ---

class MeetingInsight(pdt.BaseModel):
    """Structured insights extracted from a Google Meet recording analysis."""
    title: T.Optional[str] = pdt.Field(
        default=None,
        description="The inferred title or primary subject of the meeting based on the discussion."
    )
    summary: T.Optional[str] = pdt.Field(
        default=None,
        description="A concise summary (3-5 sentences) of the main topics discussed and key outcomes of the meeting."
    )
    questions_answers: T.Optional[list[QuestionAnswer]] = pdt.Field(
        default=None,
        description="A list of significant questions asked and their corresponding answers from the meeting."
    )
    unanswered_questions: T.Optional[list[str]] = pdt.Field(
        default=None,
        description="A list of significant questions that were asked but do not appear to have been clearly answered during the meeting.."
    )
    projects_discussed: T.Optional[list[ProjectInfo]] = pdt.Field(
        default=None,
        description="A list of specific projects discussed, including status or key points."
    )
    action_items: T.Optional[list[ActionItem]] = pdt.Field(
        default=None,
        description="A list of specific tasks or action items assigned, including owner and deadline if specified."
    )
    decisions_suggestions: T.Optional[list[DecisionSuggestion]] = pdt.Field(
        default=None,
        description="A list of key decisions made or significant suggestions proposed during the meeting."
    )
    technical_insights: T.Optional[list[str]] = pdt.Field(
       default=None,
       description="Specific mentions related technical solutions, configurations, or code snippets discussed."
    )
    key_topics_mentioned: T.Optional[list[str]] = pdt.Field(
        default=None,
        description="A list of other important topics, keywords, or themes discussed that are not captured elsewhere (e.g., specific technologies, methodologies, upcoming events, general announcements)."
    )

## Templates

In [ ]:
ANALYSIS_TEMPLATE = string.Template("""
Analyze the provided Google Meet recording thoroughly. Extract key information based on the provided schema.
Identify the meeting's purpose, summarize the discussion, list any questions/answers, pinpoint action items with owners/deadlines, note decisions/suggestions, and capture key topics mentioned.
""")

## Insights

In [ ]:
insights = {}
for key, blob in blobs.items():
    prompt = ANALYSIS_TEMPLATE.substitute()
    try:
        uri = f"gs://{blob.bucket.name}/{blob.name}"
        contents = [
            GT.Part.from_uri(file_uri=uri, mime_type="video/mp4"),
            prompt,
        ]
        response = genai_client.models.generate_content(
            model=MODEL,
            contents=contents,
            config={
                "response_mime_type": "application/json",
                "response_schema": MeetingInsight,
                "temperature": TEMPERATURE
            },
        )
        print(uri, response.usage_metadata.total_token_count)
        insights[key] = response.parsed
    except Exception as error:
        print(f"An error occurred during API call for space {key}: {error}")
len(insights)

## Markdowns

In [ ]:
markdowns = []
for key, insight in insights.items():
    parts = []
    parts.append(f"# {key.name}")
    if insight.title:
        parts.append(f"## Meeting Title")
        parts.append(insight.title)
    if insight.summary:
        parts.append(f"## Summary")
        parts.append(insight.summary)
    if insight.questions_answers:
        parts.append("## Questions & Answers")
        qa_list = []
        for qa in insight.questions_answers:
            qa_list.append(f"* **Q:** {qa.question}\n* **A:** {qa.answer}")
        parts.append("\n".join(qa_list))
    if insight.unanswered_questions:
        parts.append("## Unanswered Questions")
        uq_list = [f"* {uq}" for uq in insight.unanswered_questions]
        parts.append("\n".join(uq_list))
    if insight.projects_discussed:
        parts.append("## Projects Discussed")
        proj_list = [f"* **{proj.name}:** {proj.details}" for proj in insight.projects_discussed]
        parts.append("\n".join(proj_list))
    if insight.action_items:
        parts.append("## Action Items")
        ai_list = []
        for ai in insight.action_items:
            owner_str = f" (Owner: {ai.owner})" if ai.owner else " (Owner: Not specified)"
            deadline_str = f" (Deadline: {ai.deadline})" if ai.deadline else " (Deadline: Not specified)"
            ai_list.append(f"* **Task:** {ai.task}{owner_str}{deadline_str}")
        parts.append("\n".join(ai_list))
    if insight.decisions_suggestions:
        parts.append("## Decisions & Suggestions")
        ds_list = []
        for ds in insight.decisions_suggestions:
            details_str = f" - *Details:* {ds.details}" if ds.details else ""
            ds_list.append(f"* **[{ds.type}]** {ds.item}{details_str}")
        parts.append("\n".join(ds_list))
    if insight.technical_insights:
        parts.append("## Technical Insights")
        ti_list = [f"* {ti}" for ti in insight.technical_insights]
        parts.append("\n".join(ti_list))
    if insight.key_topics_mentioned:
        parts.append("## Key Topics")
        kt_list = [f"* {kt}" for kt in insight.key_topics_mentioned]
        parts.append("\n".join(kt_list))
    markdown = display.Markdown("\n\n".join(parts).strip())
    markdowns.append(markdown)
len(markdowns)

# EXPORTS

## Jsonlines

In [ ]:
jsonlines_path = 'video_insights.jsonlines'
with open(jsonlines_path, 'w') as file:
    for key, insight in insights.items():
        dump = insight.model_dump_json()
        file.write(dump)
        file.write('\n')
        print(dump)
if DOWNLOAD:
    files.download(jsonlines_path)

## Markdowns

In [ ]:
markdowns_path = 'video_insights.md'
with open(markdowns_path, 'w') as file:
    for markdown in markdowns:
        file.write(markdown.data)
        file.write('\n\n')
        display.display(markdown)
if DOWNLOAD:
    files.download(markdowns_path)